In [12]:
# 定义生成频繁项集的函数
from collections import defaultdict

def generate_frequent_itemsets(transactions, min_support):
    """
    生成频繁项集的函数
    :param transactions: 事务数据库，一个列表，每个元素表示一次交易，其中每个元素为一个集合，表示交易中的商品项
    :param min_support: 最小支持度，用于确定是否为频繁项集的阈值
    :return: 频繁项集，一个字典，键为频繁项集大小k，值为一个列表，表示大小为k的频繁项集
    """
    # 统计每个商品项的支持度
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            item_counts[item] = item_counts.get(item, 0) + 1

    # 生成大小为1的频繁项集
    frequent_itemsets = {1: []}
    for item, count in item_counts.items():
        if count / len(transactions) >= min_support:
            frequent_itemsets[1].append(frozenset([item]))

    # 生成大小为k的频繁项集
    k = 2
    while frequent_itemsets[k-1]:
        frequent_itemsets[k] = []
        candidates = generate_candidates(frequent_itemsets[k-1])
        item_counts = count_items(transactions, candidates)
        for itemset, count in item_counts.items():
            if count / len(transactions) >= min_support:
                frequent_itemsets[k].append(itemset)
        k += 1

    return frequent_itemsets


# 定义生成候选项集的函数
def generate_candidates(itemsets):
    """
    生成候选项集的函数
    :param itemsets: 一个列表，其中每个元素表示大小为k-1的频繁项集
    :return: 一个列表，其中每个元素表示大小为k的候选项集
    """
    candidates = []
    for i, itemset1 in enumerate(itemsets):
        for itemset2 in itemsets[i+1:]:
            if len(itemset1.intersection(itemset2)) == len(itemset1) - 1:
                candidates.append(itemset1.union(itemset2))
    return candidates


# 定义计数函数，用于计算候选项集在事务数据库中的支持度
from collections import defaultdict


def count_items(transactions, itemsets):
    """
    计算每个项集的支持度计数

    Parameters:
    -----------
    transactions: list of list
        事务记录列表
    itemsets: list of set
        候选项集列表，每个元素是一个项的集合

    Returns:
    --------
    item_counts: dict
        每个项集的支持度计数
    """
    item_counts = defaultdict(int)
    for transaction in transactions:
        for itemset in itemsets:
            if itemset.issubset(transaction):
                item_counts[itemset] += 1
    return item_counts


def apriori(transactions, min_support):
    """
    Apriori算法

    Parameters:
    -----------
    transactions: list of list
        事务记录列表
    min_support: float
        最小支持度

    Returns:
    --------
    itemsets: list of set
        所有的频繁项集
    support_dict: dict
        每个频繁项集的支持度
    """
    # 计算第一个候选项集，即每个单项的支持度计数
    itemsets = [frozenset([item]) for transaction in transactions for item in transaction]
    item_counts = count_items(transactions, itemsets)
    n_transactions = len(transactions)
    min_support_count = min_support * n_transactions

    # 存储所有频繁项集及其支持度的字典
    frequent_itemsets = {}
    support_dict = {}

    # 第一次迭代，找出频繁1项集
    frequent_itemsets[1] = {frozenset([item]): count for (item, count) in item_counts.items() if count >= min_support_count}

    # 开始迭代，直到没有更多的频繁项集
    k = 2
    while frequent_itemsets[k-1]:
        # 由上一层频繁项集生成候选项集
        candidate_itemsets = set([itemset1.union(itemset2) for itemset1 in frequent_itemsets[k-1] for itemset2 in frequent_itemsets[k-1] if len(itemset1.union(itemset2)) == k])

        # 计算候选项集的支持度计数
        item_counts = count_items(transactions, candidate_itemsets)

        # 从候选项集中找出支持度不小于min_support_count的频繁项集
        frequent_itemsets[k] = {itemset: count for (itemset, count) in item_counts.items() if count >= min_support_count}
        k += 1
    # 更新支持度字典
    support_dict.update(item_counts)
    for itemset in frequent_itemsets.values():
        for item, count in itemset.items():
            support_dict[item] = count / n_transactions

        
    return frequent_itemsets, support_dict


# 测试代码
data = [
    ['bread', 'milk'],
    ['bread', 'diaper', 'beer', 'egg'],
    ['milk', 'diaper', 'beer', 'cola'],
    ['bread', 'milk', 'diaper', 'beer'],
    ['bread', 'milk', 'diaper', 'cola'],
]

min_support = 0.4
itemsets, support_dict = apriori(data, min_support)

print('频繁项集：')
print(itemsets)

print('\n支持度字典：')
print(support_dict)


频繁项集：
{1: {frozenset({frozenset({'bread'})}): 16, frozenset({frozenset({'milk'})}): 16, frozenset({frozenset({'diaper'})}): 16, frozenset({frozenset({'beer'})}): 9, frozenset({frozenset({'cola'})}): 4}, 2: {}}

支持度字典：
{frozenset({frozenset({'bread'})}): 3.2, frozenset({frozenset({'milk'})}): 3.2, frozenset({frozenset({'diaper'})}): 3.2, frozenset({frozenset({'beer'})}): 1.8, frozenset({frozenset({'cola'})}): 0.8}


优点：

算法简单易懂，易于实现
可以处理大规模数据集
可以发现频繁项集和关联规则
缺点：

算法可能会产生大量的候选项集，导致计算开销较大
对于低支持度阈值和高维数据，可能会出现过度匹配的情况
Apriori算法只能发现项集之间的简单关系，不能处理更复杂的关系。